## Importuri necesare 

In [10]:
import os
import numpy as np
import matplotlib.pyplot as plt

## Citirea datelor

In [11]:
import pandas as pd
data_path = ''
train_data_df = pd.read_csv(os.path.join(data_path, 'train_data.csv'))



In [12]:
train_data_df

,language,text,label
0,dansk,"\nDette er et fremragende initiativ, og jeg st...",Ireland
1,dansk,"\nHr. formand, jeg er sikker på, at alle her e...",Ireland
2,dansk,"\nHr. formand, folk på den nordlige halvkugle ...",England
3,dansk,"\nHr. formand, med forbehold af nogle få ændri...",England
4,dansk,"\n\n - Hr. formand, jeg må protestere mod de...",England
...,...,...,...
41565,Nederlands,"\nMijnheer de Voorzitter, juridisch gezien is ...",England
41566,Nederlands,"\n\n . Mijnheer de Voorzitter, het is niet ...",Scotland
41567,Nederlands,"\nAls afgevaardigde van Ierland, het 'voedsele...",Ireland
41568,Nederlands,"\nMijnheer de Voorzitter, het is niet onterech...",England


In [13]:
print('Distributia etichetelor in datele de antrenare \n', train_data_df['label'].value_counts())

Distributia etichetelor in datele de antrenare 
 England     22700
Ireland     10535
Scotland     8335
Name: label, dtype: int64


In [14]:
# fiecare limba este reprezentata in mod egal
train_data_df['language'].value_counts()

dansk         8314
Deutsch       8314
español       8314
italiano      8314
Nederlands    8314
Name: language, dtype: int64

## Codificarea etichetelor din string in int

In [15]:
#Exercitiu: codificam etichetele / labels in valori cu numere intregi dela 0 la N
# codificam etichetele / labels in valori cu numere intregi dela 0 la N
etichete_unice = train_data_df['label'].unique()
print(etichete_unice)
label2id = {}
id2label = {}
for idx, eticheta in enumerate(etichete_unice):
    label2id[eticheta]=idx
    id2label[idx]=eticheta

print(label2id)
print(id2label)

['Ireland' 'England' 'Scotland']
{'Ireland': 0, 'England': 1, 'Scotland': 2}
{0: 'Ireland', 1: 'England', 2: 'Scotland'}


In [16]:
#Exercitiu: aplicati dictionarul label2id peste toate etichetele din train
labels = []
for eticheta in train_data_df['label']:
    labels.append(label2id[eticheta])
labels=np.array(labels)

print(labels[:10])

[0 0 1 1 1 0 1 1 1 2]


## Pre-procesarea datelor

- extragem informațiile necesare din text
- eliminăm semnele de punctuație
- facem tokenizare (împărțire în cuvinte)
- vezi tokenizatori in [NLTK](https://www.nltk.org/api/nltk.tokenize.html)
- vezi tokenizatori in [spaCy](https://spacy.io/)
- vezi tokenizatori precum [BPE, WordPiece, SentencePiece](https://huggingface.co/docs/transformers/tokenizer_summary)

In [17]:
import re

def proceseaza(text):
    """Functie simpla de procesare a textului.
    Sugestii:
    - cum procesati \n new lines? (vezi functia strip())
    - cum procesati empty token ''
    - puteti introduce un tokenizator din nltk
        https://www.nltk.org/api/nltk.tokenize.html
    - puteti elimina sau pastra doar stop-words
    """
    text = re.sub("[-.,;:!?\"\'\/()_*=`]", "", text)
    text_in_cuvinte = text.strip().split(' ')
    return text_in_cuvinte

# cuvintele rezultate din functia de preprocesare:
exemple_italian = train_data_df[train_data_df['language'] == 'italiano']
print(exemple_italian)


       language                                               text     label
24942  italiano  \nSi tratta di un'eccellente iniziativa e cond...   Ireland
24943  italiano  \nSignor Presidente, tutti i presenti sanno ce...   Ireland
24944  italiano  \nSignor Presidente, nel Nord del mondo si pen...   England
24945  italiano  \nSignor Presidente, fatti salvi alcuni emenda...   England
24946  italiano  \n\n   – Signor Presidente, desidero protestar...   England
...         ...                                                ...       ...
33251  italiano  \nSignor Presidente, l' obiettivo giuridico de...   England
33252  italiano  \n\n    Signor Presidente, non spetta a me tog...  Scotland
33253  italiano  \nVenendo dall'Irlanda, l'"isola del cibo", so...   Ireland
33254  italiano  \nSignor Presidente, è opportuno che io interv...   England
33255  italiano  \n\n   – E’ d’accordo il Presidente in carica ...   Ireland

[8314 rows x 3 columns]


In [18]:
text_italian = exemple_italian['text'].iloc[0]
print(proceseaza(text_italian)[:13])

['Si', 'tratta', 'di', 'uneccellente', 'iniziativa', 'e', 'condivido', 'totalmente', 'lobiettivo', 'di', 'mettere', 'in', 'rilievo']


### Aplicam functia de preprocesarea intregului set de date

In [19]:
# Exercitiu: aplicati functia pe fiecare exemplu de antrenare
data = []
for text in train_data_df['text']:
    data.append(proceseaza(text))
print(data[0])

['Dette', 'er', 'et', 'fremragende', 'initiativ', 'og', 'jeg', 'støtter', 'fuldt', 'ud', 'målet', 'om', 'at', 'fremhæve', 'idrættens', 'pædagogiske', 'rolle', 'Der', 'kan', 'opnås', 'store', 'fordele', 'ved', 'at', 'etablere', 'partnerskaber', 'mellem', 'undervisningssektorens', 'og', 'idrættens', 'organisationer\n\nSom', 'irsk', 'medlem', 'er', 'jeg', 'specielt', 'glad', 'for', 'at', 'forslaget', 'går', 'ud', 'på', 'at', 'dedikere', '2004', 'til', 'Idrættens', 'Pædagogiske', 'Dimension', 'Det', 'falder', 'sammen', 'med', 'det', 'irske', 'formandskab', 'i', 'første', 'halvdel', 'af', '2004', 'og', 'jeg', 'kan', 'forsikre', 'mine', 'kolleger', 'om', 'at', 'Irland', 'vil', 'spille', 'en', 'fremtrædende', 'rolle', 'i', 'mange', 'af', 'de', 'planlagte', 'initiativer', 'Irland', 'har', 'en', 'lang', 'tradition', 'for', 'sport', 'særligt', 'på', 'amatør', 'og', 'frivillig', 'basis\n\nVi', 'er', 'alle', 'klar', 'over', 'at', 'en', 'række', 'store', 'sportsbegivenheder', 'løber', 'af', 'stabel

## Împărțirea datelor în train, validare și test

O împărțire brutală poate fi cea în funcție de ordinea în care apar datele.
- cum poate afecat acest tip de împărțire rezultatele dacă datele sunt sortate după etichetă?

In [20]:
print(len(data))

# putem imparti datele de antrenare astfel:
# 20% date de test din total 
# 15% date de validare din ce ramane dupa ce scoatem datele de test 

nr_test = int(20/100 * len(train_data_df))
print("Nr de date de test: ", nr_test)

nr_ramase = len(data) - nr_test
nr_valid = int(15/100 * nr_ramase)
print("Nr de date de validare: ", nr_valid)

nr_train = nr_ramase - nr_valid
print("Nr de date de antrenare: ", nr_train)



41570
Nr de date de test:  8314
Nr de date de validare:  4988
Nr de date de antrenare:  28268


In [21]:
# facem impartirea in ordinea in care apar datele

train_data = data[:nr_train]
train_labels = labels[:nr_train]

valid_data = data[nr_train : nr_train + nr_valid]
valid_labels = labels[nr_train : nr_train + nr_valid]

test_data = data[nr_train + nr_valid: ]
test_labels = labels[nr_train + nr_valid:]


print(f'Nr de exemple de train {len(train_labels)}')
print(f'Nr de exemple de validare {len(valid_labels)}')
print(f'Nr de exemple de test {len(test_labels)}')

Nr de exemple de train 28268
Nr de exemple de validare 4988
Nr de exemple de test 8314


In [22]:
#Exercitiu: vedeti cum sunt distribuite etichetele in train, valid, test (incercati bincount)

## Bag of Words

În cadrul acestei secțiuni vom face numărarea aparițiilor tuturor cuvintelor din datele noastre. Pentru o evaluare justă nu ar fi indicat să includem si cuvintele din datele de test.

In [23]:
from collections import Counter

# vom folosi o structura de date nativa din python
# care functioneaza ca un dictionar care numara elementele hashable
# dintr-o colectie
ctr = Counter(['eggs', 'ham', 'eggs', 'egg'])
print(ctr['bacon'])
print(ctr['eggs'])
print(ctr)
ctr = Counter([(0,1), (0,0), (0,1)])
print(ctr[(0,0)])
print(ctr[(2,2)])


0
2
Counter({'eggs': 2, 'ham': 1, 'egg': 1})
1
0


In [24]:
ctr = Counter(data[9234])
print(ctr)
print(ctr.keys())
print(ctr.values())
# care este cel mai frecvent cuvant in data[5]?

Counter({'zu': 4, 'um': 2, 'dem': 2, 'für': 2, 'Parlament': 2, 'in': 2, 'Ich': 1, 'habe': 1, 'gegen': 1, 'diesen': 1, 'Bericht': 1, 'gestimmt': 1, 'Ausschuss': 1, 'Haushaltskontrolle': 1, 'mehr': 1, 'Zeit': 1, 'geben': 1, 'untersuchen': 1, 'ob': 1, 'seine': 1, 'Gebäude': 1, 'Straßburg': 1, 'viel': 1, 'berechnet': 1, 'wurde': 1, 'Es': 1, 'ist': 1, 'unbedingt': 1, 'erforderlich': 1, 'dass': 1, 'diese': 1, 'höchst': 1, 'unbefriedigende': 1, 'Angelegenheit': 1, 'umfassend': 1, 'untersucht': 1, 'wird': 1, 'Das': 1, 'muss': 1, 'alles': 1, 'seiner': 1, 'Macht': 1, 'Stehende': 1, 'tun': 1, 'das': 1, 'Geld': 1, 'der': 1, 'Steuerzahler': 1, 'schützen': 1})
dict_keys(['Ich', 'habe', 'gegen', 'diesen', 'Bericht', 'gestimmt', 'um', 'dem', 'Ausschuss', 'für', 'Haushaltskontrolle', 'mehr', 'Zeit', 'zu', 'geben', 'untersuchen', 'ob', 'Parlament', 'seine', 'Gebäude', 'in', 'Straßburg', 'viel', 'berechnet', 'wurde', 'Es', 'ist', 'unbedingt', 'erforderlich', 'dass', 'diese', 'höchst', 'unbefriedigende', 

### Frecventa cuvintelor din setul de antrenare

In [25]:
counter = Counter()
#TODO:
for text in train_data:
    counter.update(text)
print(counter.most_common(10))


[('de', 182565), ('la', 115758), ('der', 96800), ('que', 96475), ('en', 94038), ('die', 79963), ('at', 78111), ('a', 64202), ('i', 63742), ('og', 60776)]


### Reprezentarea datelor sub forma vectoriala

- sa presupunem ca folosim primele N cuvinte non-nule ca caracteristici pentru fiecare text


In [26]:
N = 10
cuvinte_caracteristice = []
for cuvant, frecventa in counter.most_common(N):
    if cuvant.strip():
        cuvinte_caracteristice.append(cuvant)
print(cuvinte_caracteristice)

['de', 'la', 'der', 'que', 'en', 'die', 'at', 'a', 'i', 'og']


- fiecarui cuvant îi atribuim un id în funcție de poziția pe care se află
- ordinea in care sunt stocate cheile intr-un dictionar este arbitrara 
- iar o lista este un obiect mutabil in care ordinea elementelor se poate schimba oricand
- cel mai sigur este sa construim o mapare intre cuvinte si un id care sa reprezinte pozitia in vectorul de caracteristici


In [27]:
word2id = {}
id2word = {}
for idx, cuv in enumerate(cuvinte_caracteristice):
    word2id[cuv] = idx
    id2word[idx] = cuv

print(word2id)
print(id2word)

{'de': 0, 'la': 1, 'der': 2, 'que': 3, 'en': 4, 'die': 5, 'at': 6, 'a': 7, 'i': 8, 'og': 9}
{0: 'de', 1: 'la', 2: 'der', 3: 'que', 4: 'en', 5: 'die', 6: 'at', 7: 'a', 8: 'i', 9: 'og'}


- cand trebuie sa reprezentam un text sub forma vectoriala, ne raportam doar la cuvintele cheie pe care le folosim ca caracteristici
- id-ul reprezinta pozitia in vector unde vom stoca aparitiile fiecarui cuvant

In [28]:
# 1. numaram toate cuvintele din text
ctr = Counter(train_data[1])
print(ctr)

# 2. prealocam un array care va reprezenta caracteristicel noastre
features = np.zeros(len(cuvinte_caracteristice))

# 3. umplem array-ul cu valorile obtinute din counter
# fiecare pozitie din array trebuie sa reprezinte frecventa
# aceluiasi cuvant in toate textele
for idx in range(0, len(features)):
    # obtinem cuvantul pentru pozitia idx
    cuvant = id2word[idx]
    # asignam valoarea corespunzatoare frecventei cuvantului
    features[idx] = ctr[cuvant]

print(features)
print([id2word[idx] for idx in range(0, len(features))])

Counter({'af': 11, 'er': 10, 'og': 9, 'en': 9, 'med': 6, 'vi': 6, 'et': 6, 'på': 5, 'har': 5, 'der': 5, 'at': 4, 'den': 4, 'de': 4, 'fiskeri': 4, 'det': 3, 'aftalen': 3, 'om': 3, 'for': 3, 'til': 3, 'vil': 3, 'midtvejsrevision': 3, 'i': 3, 'jeg': 2, 'her': 2, 'fiskeriaftaler': 2, 'ikke': 2, 'imod': 2, 'stærkt': 2, 'Madagaskar': 2, 'da': 2, 'meget': 2, 'Senegal': 2, 'bestande': 2, 'indflydelse': 2, 'have': 2, 'ændringsforslag': 2, 'gerne': 2, 'driver': 2, 'ikkeindustrialiseret': 2, 'Kommissionen': 2, 'Hr': 1, 'formand': 1, 'sikker': 1, 'alle': 1, 'klar': 1, 'over': 1, 'De': 1, 'Grønne': 1, 'grundlæggende': 1, 'problemer': 1, 'Den': 1, 'Europæiske': 1, 'Unions': 1, 'behandling': 1, 'Vi': 1, 'principielt': 1, 'Men': 1, 'protesterer': 1, 'mod': 1, 'måde': 1, 'hvorpå': 1, 'forhandles': 1, 'grundlag': 1, 'hvilke': 1, 'forhandlingerne': 1, 'finder': 1, 'sted\n\nDa': 1, 'udarbejdede': 1, 'Madagaskarbetænkningen': 1, 'foretog': 1, 'detaljeret': 1, 'undersøgelse': 1, 'fiskeriaftalen': 1, 'fandt'

### Punem totul cap la cap sub forma de functii

In [29]:
def count_most_common(how_many, texte_preprocesate):
    """Functie care returneaza cele mai frecvente cuvinte.
    """
    counter = Counter()
    #TODO:
    for text in texte_preprocesate:
        counter.update(text)
    cuvinte_caracteristice = []
    for cuvant, frecventa in counter.most_common(how_many):
        if cuvant.strip():
            cuvinte_caracteristice.append(cuvant)
    return cuvinte_caracteristice

In [30]:
def build_id_word_dicts(cuvinte_caracteristice):
    '''Dictionarele word2id si id2word garanteaza o ordine
    pentru cuvintele caracteristice.
    '''
    word2id = {}
    id2word = {}
    for idx, cuv in enumerate(cuvinte_caracteristice):
        word2id[cuv] = idx
        id2word[idx] = cuv

    return word2id, id2word

In [31]:
def featurize(text_preprocesat, id2word):
    """Pentru un text preprocesat dat si un dictionar
    care mapeaza pentru fiecare pozitie ce cuvant corespunde,
    returneaza un vector care reprezinta
    frecventele fiecarui cuvant.
    """
    ctr = Counter(text_preprocesat)
    features = np.zeros(len(id2word))
    for idx in range(0, len(features)):
        cuvant = id2word[idx]
        features[idx] = ctr[cuvant]
    return features

In [32]:
def featurize_multi(texte, id2word):
    '''Pentru un set de texte preprocesate si un dictionar
    care mapeaza pentru fiecare pozitie ce cuvant corespunde,
    returneaza matricea trasaturilor tuturor textelor.
    '''
    all_features = []
    for text in texte:
        all_features.append(featurize(text, id2word))
    return np.array(all_features)

In [40]:
cuvinte_caracteristice = count_most_common(25, train_data)
print(len(cuvinte_caracteristice))
word2id, id2word = build_id_word_dicts(cuvinte_caracteristice)

X_train = featurize_multi(train_data, id2word)
X_valid = featurize_multi(valid_data, id2word)
X_test = featurize_multi(test_data, id2word)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)


25
(28268, 25)
(4988, 25)
(8314, 25)


In [34]:
from sklearn.metrics import accuracy_score

In [35]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, train_labels)
vpreds = neigh.predict(X_valid)
tpreds = neigh.predict(X_test)

print(accuracy_score(valid_labels, vpreds))
print(accuracy_score(test_labels, tpreds))

0.4264234161988773
0.43144094298773156


In [36]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, train_labels)
vpreds = clf.predict(X_valid)
tpreds = clf.predict(X_test)

print(accuracy_score(valid_labels, vpreds))
print(accuracy_score(test_labels, tpreds))

0.2660384923817161
0.22275679576617755


In [37]:
from sklearn import svm

model = svm.LinearSVC(C=1)

model.fit(X_train, train_labels)
vpreds = model.predict(X_valid)
tpreds = model.predict(X_test)

print(accuracy_score(valid_labels, vpreds))
print(accuracy_score(test_labels, tpreds))

0.506214915797915
0.32078421938898244


C:\Users\andrr\.conda\envs\LabAI\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Exemplu de lucru cu stop words in nltk

In [39]:
# luat in calcul cuvintele functionale, pot creste sau scadea acuratetea
# in cazul de fata, cuvintele functionale (stop words) indica elemente de gramatica
# care sunt specifice textelor de la Scotieni, Englezi, Irlandezi
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
print(stopwords.words('french'))

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andrr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
